In [6]:
#Projeto: práticas de ETL 
#Fonte de dados: arquivo .csv de ocorrencias de acidentes/incidentes aéreos ocorridos no Brasil
#no periode entre 2012 e 2021 (https://www2.fab.mil.br/cenipa/)

In [8]:
#Utilização das bibliotecas pandas e pandera
!pip install pandas
!pip install pandera

     |████████████████████████████████| 197 kB 5.1 MB/s 
     |████████████████████████████████| 10.9 MB 34.2 MB/s 


In [ ]:
#Carregamento e limpeza inicial dos dados
#Após observações gerais no banco de dados, excluiu-se algumas colunas que não 
#seriam interessantes para esta análise e observou-se a existentencia de missing values. 
#Tratou-se os missing values substituindo-os por NaN.

In [9]:
import pandas as pd
import pandera as pa
valores_ausentes=['**','###!', '####', '****', '*****', 'NULL']
df=pd.read_csv("ocorrencia.csv",sep=";",parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes)

In [ ]:
#Para a etapa de verificação dos dados foi utilizada a função da DataFrameschema da 
#biblioteca pandera. Verificou-se a congruencia dos tipos de dados, formato
#visualização da data, e também a apresentação das siglas dos estados.

In [10]:
#Definição do schema
schema=pa.DataFrameSchema(
  columns={"codigo_ocorrencia":pa.Column(pa.Int),
          "codigo_ocorrencia2":pa.Column(pa.Int),
          "ocorrencia_classificacao":pa.Column(pa.String),
          "ocorrencia_cidade":pa.Column(pa.String),
          "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
          "ocorrencia_aerodromo":pa.Column(pa.String,nullable=True),
          "ocorrencia_dia":pa.Column(pa.DateTime),
          "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$'), nullable=True),
          "total_recomendacoes":pa.Column(pa.Int)})

In [11]:
#Validação do DataFrame
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3
2,45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0
...,...,...,...,...,...,...,...,...,...
5162,80458,80458,ACIDENTE,JATAÍ,GO,NaN,2021-12-30,20:30:00,0
5163,80452,80452,ACIDENTE,MARACAÍ,SP,NaN,2021-12-31,09:30:00,0
5164,80457,80457,INCIDENTE GRAVE,NOVO HAMBURGO,RS,SSNH,2021-12-31,11:59:00,0
5165,80460,80460,INCIDENTE,CURITIBA,PR,SBBI,2021-12-31,15:12:00,0


In [12]:
#Verificando como apresentam-se os tipos dos dados
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [13]:
#Vamos criar uma nova coluna, chamada "ocorrencia_dia_hora" do dataframe, que apresente, junto, a data e a hora das ocorrências
df['ocorrencia_dia_hora'] = pd.to_datetime(df.ocorrencia_dia.astype(str) + ' ' +df.ocorrencia_hora)

In [14]:
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,52242,52242,INCIDENTE,PORTO ALEGRE,RS,SBPA,2012-01-05,20:27:00,0,2012-01-05 20:27:00
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,2012-01-06 13:44:00
2,45333,45333,ACIDENTE,VIAMÃO,RS,NaN,2012-01-06,13:00:00,0,2012-01-06 13:00:00
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0,2012-01-06 17:00:00
4,45407,45407,ACIDENTE,SÃO SEPÉ,RS,NaN,2012-01-06,16:30:00,0,2012-01-06 16:30:00


In [15]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
ocorrencia_dia_hora         datetime64[ns]
dtype: object

In [18]:
#Localização de dados específicos com data e hora específicos
filtro1=df.ocorrencia_dia_hora >= '2015-12-03 11:00:00'
filtro2=df.ocorrencia_dia_hora <= '2015-12-08 13:00:00'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
2307,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
2308,53625,53625,ACIDENTE,TRINDADE,GO,NaN,2015-12-06,13:10:00,3,2015-12-06 13:10:00
2309,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
2311,53629,53629,INCIDENTE GRAVE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00
2313,60636,60636,INCIDENTE,CAXIAS DO SUL,RS,SBCX,2015-12-08,13:00:00,0,2015-12-08 13:00:00


In [21]:
#Agrupamento de dados e criação de novos dataframes
#Ocorrencias do ano do mês de março do ano de 2015
filtro1=df.ocorrencia_dia.dt.year == 2015
filtro2=df.ocorrencia_dia.dt.month == 3
df201503=df.loc[filtro1 & filtro2]
df201503

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
1960,53183,53183,ACIDENTE,AMAPORÃ,PR,NaN,2015-03-02,23:00:00,0,2015-03-02 23:00:00
1961,53120,53120,ACIDENTE,CHAVES,PA,SNXW,2015-03-04,13:30:00,0,2015-03-04 13:30:00
1962,53109,53109,ACIDENTE,CAMPO GRANDE,MS,SSIE,2015-03-05,13:50:00,0,2015-03-05 13:50:00
1963,53112,53112,ACIDENTE,MOGI GUAÇU,SP,NaN,2015-03-06,21:00:00,1,2015-03-06 21:00:00
1964,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
1965,53167,53167,INCIDENTE,MARABÁ,PA,SBMA,2015-03-10,17:33:00,0,2015-03-10 17:33:00
1966,53596,53596,INCIDENTE,ITAPEMA,SC,NaN,2015-03-10,21:35:00,0,2015-03-10 21:35:00
1967,53149,53149,ACIDENTE,TABATINGA,AM,SBTT,2015-03-11,23:25:00,0,2015-03-11 23:25:00
1968,53148,53148,ACIDENTE,ARARAS,SP,SDEH,2015-03-12,12:40:00,0,2015-03-12 12:40:00
1969,53153,53153,INCIDENTE,PORTO ALEGRE,RS,SBPA,2015-03-13,15:45:00,0,2015-03-13 15:45:00


In [22]:
#Buscando o número de ocorrencias deste periodo, com filtragem pela classificação da ocorrência
df201503.groupby(['ocorrencia_classificacao']).size().sort_values(ascending=False)

ocorrencia_classificacao
INCIDENTE          17
ACIDENTE           15
INCIDENTE GRAVE     5
dtype: int64

In [24]:
#Agrupamento de dados e criação de novos dataframes
#Ocorrencias na região sudeste no ano de 2012
filtro1=df.ocorrencia_dia.dt.year == 2012
filtro2=df.ocorrencia_uf.isin(['SP','MG','RG','ES'])
dfsudeste2012=df.loc[filtro1 & filtro2]
dfsudeste2012.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
1,45331,45331,ACIDENTE,GUARULHOS,SP,SBGR,2012-01-06,13:44:00,3,2012-01-06 13:44:00
3,45401,45401,ACIDENTE,SÃO SEBASTIÃO,SP,NaN,2012-01-06,17:00:00,0,2012-01-06 17:00:00
5,52243,52243,INCIDENTE,UBATUBA,SP,NaN,2012-01-06,14:30:00,0,2012-01-06 14:30:00
6,50713,50713,INCIDENTE GRAVE,CAMPINAS,SP,SDAI,2012-01-07,18:15:00,0,2012-01-07 18:15:00
8,45391,45391,ACIDENTE,CONCEIÇÃO DAS ALAGOAS,MG,NaN,2012-01-08,16:00:00,0,2012-01-08 16:00:00


In [25]:
#Número de acidentes na região sudeste, agrupados pela classificação
dfsudeste2012.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE            71
INCIDENTE          135
INCIDENTE GRAVE     26
dtype: int64

In [26]:
#Número de acidentes na região sudeste, agrupados pela classificação e estados
dfsudeste2012.groupby(['ocorrencia_classificacao','ocorrencia_uf']).size()

ocorrencia_classificacao  ocorrencia_uf
ACIDENTE                  ES                1
                          MG               21
                          SP               49
INCIDENTE                 ES               11
                          MG               37
                          SP               87
INCIDENTE GRAVE           MG                9
                          SP               17
dtype: int64

In [27]:
#Avaliando as ocorrencias por cidade da região sudeste
dfsudeste2012.groupby(['ocorrencia_cidade']).size().sort_values(ascending=False)

ocorrencia_cidade
SÃO PAULO               31
BELO HORIZONTE          25
CAMPINAS                17
GUARULHOS               16
JUNDIAÍ                 15
                        ..
GUARUJÁ                  1
GUARATINGUETÁ            1
GOVERNADOR VALADARES     1
FRANCO DA ROCHA          1
ITUIUTABA                1
Length: 84, dtype: int64

In [30]:
#Verificando o total de recomendações por cidade
filtro=dfsudeste2012.total_recomendacoes > 0 
dfsudeste2012.loc[filtro].groupby(['ocorrencia_cidade']).total_recomendacoes.sum().sort_values()

ocorrencia_cidade
TARUMIRIM                1
CAMPINAS                 1
JUIZ DE FORA             1
DRACENA                  1
ELDORADO                 1
ITU                      1
SANTA BÁRBARA D'OESTE    2
PARÁ DE MINAS            2
LENÇÓIS PAULISTA         2
JUNDIAÍ                  2
ITÁPOLIS                 2
ITANHAÉM                 2
BEBEDOURO                2
ESPINOSA                 2
CAMPO FLORIDO            2
GUARUJÁ                  3
CATANDUVA                3
JUQUIÁ                   3
BELO HORIZONTE           3
BRAGANÇA PAULISTA        4
GUARULHOS                4
BOITUVA                  7
SÃO PAULO                8
Name: total_recomendacoes, dtype: int64

In [28]:
#Verificando as ocorrencias de São Paulo e o número de recomendações
filtro1=dfsudeste2012.ocorrencia_cidade == 'SÃO PAULO'
filtro2=dfsudeste2012.total_recomendacoes > 0 
dfsudeste2012.loc[filtro1].total_recomendacoes.sum()

8

In [31]:
#Total de recomendações por cidade, mas que exista recomendações, por mês
filtro=dfsudeste2012.total_recomendacoes > 0 
dfsudeste2012.loc[filtro].groupby(['ocorrencia_cidade',dfsudeste2012.ocorrencia_dia.dt.month]).total_recomendacoes.sum()

ocorrencia_cidade      ocorrencia_dia
BEBEDOURO              7                 2
BELO HORIZONTE         8                 3
BOITUVA                7                 7
BRAGANÇA PAULISTA      4                 4
CAMPINAS               8                 1
CAMPO FLORIDO          2                 2
CATANDUVA              10                3
DRACENA                5                 1
ELDORADO               1                 1
ESPINOSA               7                 2
GUARUJÁ                4                 3
GUARULHOS              1                 3
                       4                 1
ITANHAÉM               4                 2
ITU                    3                 1
ITÁPOLIS               12                2
JUIZ DE FORA           7                 1
JUNDIAÍ                4                 2
JUQUIÁ                 10                3
LENÇÓIS PAULISTA       5                 2
PARÁ DE MINAS          12                2
SANTA BÁRBARA D'OESTE  8                 2
SÃO PAULO       

In [33]:
#Podemos verificar essa informação
filtro1=dfsudeste2012.total_recomendacoes > 0 
dfsudeste2012.loc[filtro1&filtro2,['ocorrencia_cidade','total_recomendacoes','ocorrencia_dia']]

,ocorrencia_cidade,total_recomendacoes,ocorrencia_dia
1,GUARULHOS,3,2012-01-06
13,ELDORADO,1,2012-01-11
46,CAMPO FLORIDO,2,2012-02-02
140,ITU,1,2012-03-24
165,BRAGANÇA PAULISTA,4,2012-04-11
173,ITANHAÉM,2,2012-04-14
181,JUNDIAÍ,2,2012-04-20
187,GUARULHOS,1,2012-04-24
191,GUARUJÁ,3,2012-04-26
200,LENÇÓIS PAULISTA,2,2012-05-01
